In [ ]:
import src.dataloader
import torch
import pathlib

device = torch.device("cuda")

dataloader = src.dataloader.load_train_data(pathlib.Path("C:/Users/imeho/Documents/DataSets/InstantPINF/ScalarReal"),
                                            "train",
                                            device=device)
image_batch = next(iter(dataloader))

print(f'image device: {image_batch["image"].device}')
memory_image_cuda = image_batch['image'].element_size() * image_batch['image'].numel()
print(f'Memory of image: {memory_image_cuda / 1024 / 1024:.2f} MB')

In [ ]:
from nerfstudio.model_components.ray_samplers import UniformSampler
from nerfstudio.model_components.scene_colliders import NearFarCollider

collider = NearFarCollider(near_plane=1.1, far_plane=1.5).to(device)
sampler_uniform = UniformSampler(num_samples=192).to(device)

In [ ]:
import src.encoder
import taichi as ti
import numpy as np

ti.init(arch=ti.cuda)
xyzt_encoder = src.encoder.HashEncoderHyFluid(
    min_res=np.array([16, 16, 16, 16]),
    max_res=np.array([256, 256, 256, 128]),
    num_scales=16,
    max_params=2 ** 19,
)
xyzt_encoder.to(device)

In [ ]:
from nerfstudio.field_components.mlp import MLP
import src.radam

mlp_base = MLP(
    in_dim=xyzt_encoder.num_scales * xyzt_encoder.features_per_level,
    num_layers=2,
    layer_width=64,
    out_dim=1,
    out_activation=torch.nn.ReLU(),
)
mlp_base.to(device)
learned_rgb = torch.nn.Parameter(torch.tensor([0.0], device=device))

grad_vars = list(mlp_base.parameters()) + [learned_rgb]
embedding_params = list(xyzt_encoder.parameters())

optimizer = src.radam.RAdam([
    {'params': grad_vars, 'weight_decay': 1e-6},
    {'params': embedding_params, 'eps': 1e-15}
], lr=0.01, betas=(0.9, 0.99))

In [ ]:
raw2alpha = lambda raw, dists, act_fn=torch.nn.functional.relu: 1. - torch.exp(-act_fn(raw) * dists)

In [ ]:
import nerfstudio.data.pixel_samplers
import nerfstudio.model_components.ray_generators

ps = nerfstudio.data.pixel_samplers.PixelSamplerConfig(num_rays_per_batch=1024).setup()
rg = nerfstudio.model_components.ray_generators.RayGenerator(dataloader.dataset.cameras).to(device)

In [ ]:
def image_idx_to_frame(image_indices, all_frames):
    cumulative_frames = torch.cumsum(torch.tensor([0] + all_frames[:-1], device=image_indices.device), dim=0)
    result = []
    for image_idx in image_indices:
        video_idx = torch.searchsorted(cumulative_frames, image_idx, right=True) - 1
        frame_in_video = image_idx - cumulative_frames[video_idx]
        result.append(frame_in_video.item())
    return torch.tensor(result, device=image_indices.device)

In [ ]:
import nerfstudio.model_components.losses
import tqdm
import time
import matplotlib.pyplot as plt

# 初始化 MSE 损失函数
rgb_loss = nerfstudio.model_components.losses.MSELoss()

# 定义列表存储时间
encoding_times = []
loss_calc_times = []
optimizer_times = []

# 主循环
for i in tqdm.tqdm(range(100)):
    # 记录编码时间
    start = time.time()
    batch = ps.sample(image_batch)
    ray_bundle = rg(batch['indices'])
    collider(ray_bundle)
    ray_samples_uniform = sampler_uniform(ray_bundle)
    positions = ray_samples_uniform.frustums.get_positions()

    frames = image_idx_to_frame(image_indices=batch['indices'][:, 0],
                                all_frames=dataloader.dataset.metadata['all_frames'])
    frames_expanded = frames.view(positions.shape[0], 1, 1).expand(-1, positions.shape[1], -1)
    xyzt = torch.cat((positions, frames_expanded), dim=-1)
    xyzt_flat = xyzt.reshape(-1, 4)
    xyzt_encoded = xyzt_encoder(xyzt_flat)
    end = time.time()
    encoding_times.append(end - start)  # 存储编码耗时

    # 记录损失计算时间
    start = time.time()
    raw_flat = mlp_base(xyzt_encoded)
    raw = raw_flat.reshape(xyzt.shape[0], xyzt.shape[1], raw_flat.shape[-1])
    dists = ray_samples_uniform.deltas
    rgb = torch.ones(3, device=device) * (0.6 + torch.tanh(learned_rgb) * 0.4)
    alpha = raw2alpha(raw[..., -1], dists[..., -1])
    weights = alpha * torch.cumprod(torch.cat([torch.ones((alpha.shape[0], 1), device=device), 1. - alpha + 1e-10], -1),
                                    -1)[:, :-1]
    rgb_map = torch.sum(weights[..., None] * rgb, -2)
    loss = rgb_loss(rgb_map, batch['image'])
    end = time.time()
    loss_calc_times.append(end - start)  # 存储损失计算耗时

    # 记录优化时间
    start = time.time()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    new_lrate = 5e-4 * (0.1 ** (i / 250))
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lrate

    end = time.time()
    optimizer_times.append(end - start)  # 存储优化耗时

# 绘制折线图
plt.figure(figsize=(10, 6))
plt.plot(encoding_times, label='Encoding Time')
plt.plot(loss_calc_times, label='Loss Calculation Time')
plt.plot(optimizer_times, label='Optimizer Time')
plt.xlabel('Iteration')
plt.ylabel('Time (seconds)')
plt.title('Time Analysis per Operation')
plt.legend()
plt.grid(True)
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

Encoding took 0.23 seconds


  1%|          | 1/100 [00:00<00:56,  1.77it/s]

Loss calculation took 0.34 seconds
optimizer 0 took 0.00 seconds


  2%|▏         | 2/100 [00:14<13:56,  8.54s/it]

...

Encoding took 14.02 seconds
Loss calculation took 0.03 seconds
optimizer 28 took 0.05 seconds


 30%|███       | 30/100 [06:39<16:08, 13.83s/it]

Encoding took 13.82 seconds
Loss calculation took 0.05 seconds
optimizer 29 took 0.00 seconds


 31%|███       | 31/100 [06:53<15:55, 13.85s/it]

Encoding took 13.88 seconds
Loss calculation took 0.03 seconds
optimizer 30 took 0.00 seconds
